In [32]:
import mlrun

project_name = "jw-proj"
project = mlrun.get_or_create_project(project_name)

> 2025-03-14 15:54:07,484 [info] Project loaded successfully: {"project_name":"jw-proj"}


In [33]:
project.spec.artifact_path="v3io:///projects/jw-proj/artifacts/"
project.save()

## Function that will fail

In [34]:
%%writefile ./func-error.py
def func(i):
    i = i+1
    raise ValueError()
    return i

Overwriting ./func-error.py


In [35]:
func = project.set_function(
    func="./func-error.py",
    name="func-error",
    kind="job",
    handler="func",
    image="mlrun/mlrun",
)

In [36]:
project.build_function(func)

The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.9.0.


BuildStatus(ready=True, outputs={'image': 'mlrun/mlrun'})

## Job with status error

In [37]:
func.run(params={"i": 1}, watch=False)

> 2025-03-14 15:54:38,951 [info] Storing function: {"db":"https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com","name":"func-error-func","uid":"fd8c4f5f097e4201b5e25d19938dc4a2"}
> 2025-03-14 15:54:45,109 [info] Job is running in the background, pod: func-error-func-qfj8t


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
jw-proj,...8dc4a2,0,Mar 14 13:54:45,NaT,running,,func-error-func,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.8.0-rc41mlrun/client_python_version=3.9.13,,i=1,


> 2025-03-14 15:54:45,125 [info] Run execution finished: {"name":"func-error-func","status":"running"}


## Workflow with status error

In [38]:
%%writefile ./workflow-error.py


import mlrun

def pipeline():
    
    step_1 = mlrun.run_function("func-error",params={"i":1},outputs=["return"])
    step_2 = mlrun.run_function("func-error",params={"i":step_1.outputs["return"]},outputs=["return"])
    step_3 = mlrun.run_function("func-error",params={"i":step_2.outputs["return"]})

Overwriting ./workflow-error.py


In [52]:
project.set_workflow("main-error", "./workflow-error.py")
project.save()

In [60]:
project.run("main-error")

> 2025-03-14 16:11:09,197 [info] Submitting remote workflow: {"project_name":"jw-proj","schedule":null,"workflow_engine":"local"}
> 2025-03-14 16:11:17,594 [error] Workflow run failed: {"exc_info":["Traceback (most recent call last):\n","  File \"c:\\Users\\tetiana.kharchuk\\test-notebooks-ui\\mlrun-env\\lib\\site-packages\\mlrun\\errors.py\", line 96, in raise_for_status\n    response.raise_for_status()\n","  File \"c:\\Users\\tetiana.kharchuk\\test-notebooks-ui\\mlrun-env\\lib\\site-packages\\requests\\models.py\", line 1024, in raise_for_status\n    raise HTTPError(http_error_msg, response=self)\n","requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com/api/v1/projects/jw-proj/workflows/main-error/submit\n","\nThe above exception was the direct cause of the following exception:\n\n","Traceback (most recent call last):\n","  File \"c:\\Users\\tetiana.kharchuk\\test-notebooks-ui\\mlrun-env\\lib\\site-packages

None

## Function that will be succeeded

In [20]:
%%writefile ./func-compl.py
import time
def func(i):
    i = i+1
    time.sleep(20)
    return i

Writing ./func-compl.py


In [21]:
func_compl = project.set_function(
    func="./func-compl.py",
    name="func-compl",
    kind="job",
    handler="func",
    image="mlrun/mlrun",
)

In [22]:
project.build_function(func_compl)

The `overwrite_build_params` parameter default will change from 'False' to 'True' in 1.9.0.


BuildStatus(ready=True, outputs={'image': 'mlrun/mlrun'})

## Job with status completed

In [23]:
func_compl.run(params={"i": 1}, watch=False)

> 2025-03-14 15:48:43,798 [info] Storing function: {"db":"https://mlrun-api.default-tenant.app.vmdev54.lab.iguazeng.com","name":"func-compl-func","uid":"5dd21a315e234e0ebf0beaf28287a836"}
> 2025-03-14 15:48:49,854 [info] Job is running in the background, pod: func-compl-func-rc68x


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
jw-proj,...87a836,0,Mar 14 13:48:49,NaT,running,,func-compl-func,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.8.0-rc41mlrun/client_python_version=3.9.13,,i=1,


> 2025-03-14 15:48:49,869 [info] Run execution finished: {"name":"func-compl-func","status":"running"}


## Workflow with status completed

In [24]:
%%writefile ./workflow-compl.py


import mlrun

def pipeline():
    
    step_1 = mlrun.run_function("func-compl",params={"i":1},outputs=["return"])
    step_2 = mlrun.run_function("func-compl",params={"i":step_1.outputs["return"]},outputs=["return"])
    step_3 = mlrun.run_function("func-compl",params={"i":step_2.outputs["return"]})

Writing ./workflow-compl.py


In [25]:
project.set_workflow("main-compl", "./workflow-compl.py")
project.save()

In [61]:
# project.run("main-compl", watch=False)
project.run(
    workflow_path="./workflow-compl.py",
    watch=False,
)

> 2025-03-14 16:21:06,934 [info] Storing function: {"db":null,"name":"func-compl-func","uid":"3d41782a48d14776a3fe4f2c3a18b86a"}


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
jw-proj,...18b86a,0,Mar 14 14:21:43,NaT,completed,run,func-compl-func,workflow=0f325b3094f142459530bccd7af84043v3io_user=normal-userkind=localowner=normal-userhost=LWO1-LHP-A04834,,i=1,return=2


> 2025-03-14 16:22:20,501 [info] Run execution finished: {"name":"func-compl-func","status":"completed"}
> 2025-03-14 16:22:20,503 [info] Storing function: {"db":null,"name":"func-compl-func","uid":"7d0c046c4cb14066bf2f830c872d4017"}


KeyboardInterrupt: 

## Job with status aborted + wf with status error

In [28]:
# project.run("main-compl")

In [29]:
# get running job uid from Overwiew or YAML
# uid = "d4df60806d6a4e85bfb6b5c137294266"

In [30]:
# db = mlrun.get_run_db()
# db.abort_run(uid, project_name)